In [1]:
import importlib
import sys
from urllib.request import urlretrieve

# Imports.
import numpy as np
import meshcat
from ipywidgets import interact, FloatSlider, ToggleButton
from IPython.display import display

import pydrake.all
from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder, PlanarSceneGraphVisualizer, SceneGraph, Simulator)
from pydrake.systems.jupyter_widgets import WidgetSystem
from pydrake.examples.pendulum import PendulumGeometry, PendulumPlant

import underactuated
from underactuated.jupyter import AdvanceToAndVisualize, SetupMatplotlibBackend, running_as_notebook
import underactuated.meshcat_utils as mutil
from pydrake.examples.pendulum import PendulumGeometry, PendulumPlant, PendulumInput
from pydrake.all import Linearize, LinearQuadraticRegulator,VectorSystem
import math
from pydrake.all import Jacobian, MathematicalProgram, Solve, Variables, Polynomial
import numpy as np

In [3]:
### Prepare non-linear system

# non-linear dyamics
prog = MathematicalProgram()
x = prog.NewIndeterminates(4, "x")
x1 = x[0]
x2 = x[1]
x3 = x[2]
x4 = x[3]

r = [0, 0, pi, 0]  # reference
f1 = x2
f2 = (2233260595080309*x2)/1407374883553280 - (3253141686419969*x3)/87960930222080 - (1360884314958167*x4)/87960930222080 + (3253141686419969*pi)/87960930222080 + (10^(1/2)*x1)/5 - (2233260595080309*x2*(x3 - pi)^2)/7036874417766400 + (1360884314958167*x4*(x3 - pi)^2)/439804651110400 - (2*x4^2*(x3 - pi))/5 + (8000206454818307*(x3 - pi)^3)/1319413953331200 - (10^(1/2)*x1*(x3 - pi)^2)/25
f3 = x4
f4 = (2233260595080309*x2)/2814749767106560 - (2373532384199169*x3)/175921860444160 - (1360884314958167*x4)/175921860444160 + (2373532384199169*pi)/175921860444160 + (10^(1/2)*x1)/10 - (15632824165562163*x2*(x3 - pi)^2)/28147497671065600 + (9526190204707169*x4*(x3 - pi)^2)/1759218604441600 - (x4^2*(x3 - pi))/5 + (20133163898277383*(x3 - pi)^3)/1759218604441600 - (7*10^(1/2)*x1*(x3 - pi)^2)/100
 
fn = [f1,f2,f3,f4]

In [4]:
### Lyapunov analysis, fix V and find Lagrange multiplier.

# cost-to-go of LQR as Lyapunov candidate
V = (1072252633462743*x1*x2)/35184372088832 - (3257132805361133*x1*x4)/35184372088832 - (2151747033632871*x1*(x3 - pi))/8796093022208 + (3531095044578367*x1^2)/140737488355328, (1072252633462743*x1*x2)/35184372088832 - (3496410051741759*x2*x4)/17592186044416 - (286527315739597*x2*(x3 - pi))/549755813888 + (3849400638744835*x2^2)/70368744177664, (3289273979748465*x4*(x3 - pi))/1099511627776 - (286527315739597*x2*(x3 - pi))/549755813888 - (2151747033632871*x1*(x3 - pi))/8796093022208 + (4208136145416213*(x3 - pi)^2)/549755813888, (3289273979748465*x4*(x3 - pi))/1099511627776 - (3496410051741759*x2*x4)/17592186044416 - (3257132805361133*x1*x4)/35184372088832 + (5150415813266293*x4^2)/4398046511104
Vdot = Jacobian([V], x).dot(fn)[0]

# Define the Lagrange multiplier.
lambda_ = prog.NewSosPolynomial(Variables(x), 2)[0].ToExpression()

# Optimization setup
rho = 1.
prog.AddSosConstraint(-Vdot + lambda_*(V-rho))
prog.AddSosConstraint(V)
prog.AddSosConstraint(lambda_)

result = Solve(prog)

# Print result
if (result.is_success()):
    print(f"Verified that {str(V)} < {rho} is in the region of attraction.")
    #print(Polynomial(result.GetSolution(lambda_)))
else:
    print("failed")

Verified that ((x(1) * (0.60649156397967863 * x(1) + 2.6852527630200029 * (-3.1415926535897931 + x(0)))) + ((-3.1415926535897931 + x(0)) * (2.6852527630200025 * x(1) + 15.232066986722771 * (-3.1415926535897931 + x(0))))) < 22.0 is in the region of attraction.


In [5]:
x0 = np.pi*2/3
x1 = 1
FV = ((x1* (0.60649156397967863 * x1+ 2.6852527630200029 * (-3.1415926535897931 + x0))) + ((-3.1415926535897931 + x0) * (2.6852527630200025 * x1+ 15.232066986722771 * (-3.1415926535897931 + x0)))) < 22.0
print(f"under IV x=[{x0}, {x1}], it is {FV} in RoA")

under IV x=[2.0943951023931953, 1], it is True in RoA
